In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, clear_output

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

def download_html_table(url):
    try:
        html = urlopen(url)
    except (HTTPError, URLError) as e:
        print(f"Error cargando {url}: {e}")
        return None

    soup = BeautifulSoup(html.read(), "html.parser")
    table = soup.find("table", {"class": "medias"})
    
    if not table:
        print(f"No se encontró la tabla en: {url}")
        return None

    titles = [th.get_text(strip=True) for th in table.findAll("th")]
    rows = table.findAll("tr")[1:]
    
    data = []
    for row in rows:
        cells = row.findAll("td")
        row_data = []
        for cell in cells:
            text = cell.get_text(strip=True).replace(",", ".")
            try:
                value = float(text) if '-' not in text and text else 0
            except ValueError:
                value = text
            row_data.append(value)
        if row_data:
            data.append(row_data)

    df = pd.DataFrame(data=data, columns=titles[:len(data[0])])
    return df

def get_climate_data_for_years(station_code, start_year, end_year):
    all_data = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            url = f"https://www.tutiempo.net/clima/{month:02d}-{year}/ws-{station_code}.html"
            print(f"Descargando: {url}")
            df = download_html_table(url)
            if df is not None:
                df["Año"] = year
                df["Mes"] = month
                all_data.append(df)
    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

def cargar_y_mostrar_datos(station_code, anio_inicio, anio_fin):
    clear_output()
    df_clima = get_climate_data_for_years(station_code, anio_inicio, anio_fin)

    if df_clima.empty:
        print("No se encontraron datos para el rango seleccionado.")
        return

    df_clima.rename(columns=lambda x: x.strip().capitalize(), inplace=True)

    for col in ["T", "Tm", "Tn", "Humedad", "Velocidad del viento", "Presión"]:
        if col in df_clima.columns:
            df_clima[col] = pd.to_numeric(df_clima[col], errors="coerce")

    # Visualización: Temperatura promedio mensual
    df_grouped = df_clima.groupby(["Año", "Mes"])["T"].mean().reset_index()

    plt.plot(df_grouped["Mes"], df_grouped["T"], marker="o")
    plt.title("Temperatura Promedio Mensual")
    plt.xlabel("Mes")
    plt.ylabel("Temperatura (°C)")
    plt.grid(True)
    plt.show()

    # Visualización: Boxplot de temperatura por año
    sns.boxplot(data=df_clima, x="Año", y="T")
    plt.title("Distribución de Temperatura por Año")
    plt.show()

    # Exportar a CSV
    archivo = f"clima_bogota_{anio_inicio}_{anio_fin}.csv"
    df_clima.to_csv(archivo, index=False)
    print(f"Archivo exportado exitosamente: {archivo} 🎉")

# Widgets interactivos
anio_inicio = widgets.IntSlider(value=2022, min=2000, max=2025, description='Año inicio')
anio_fin = widgets.IntSlider(value=2023, min=2000, max=2025, description='Año fin')
boton = widgets.Button(description="Cargar datos")

ui = widgets.VBox([anio_inicio, anio_fin, boton])
display(ui)

def on_button_clicked(b):
    cargar_y_mostrar_datos("802220", anio_inicio.value, anio_fin.value)

boton.on_click(on_button_clicked)
